# Notebook 02: Exploration des Stratégies RAG

Ce notebook se concentre sur l'exploration des capacités de notre `RetrievalEngine` pour récupérer des informations pertinentes à partir de la base de connaissances MongoDB que nous avons peuplée. Nous allons tester la recherche vectorielle simple et la recherche avec filtres de métadonnées.

**Prérequis :**
* Avoir exécuté le notebook `00_setup_environment.ipynb` pour configurer l'environnement et les variables d'environnement (fichier `.env`).
* Avoir exécuté le notebook `01_data_ingestion_and_embedding.ipynb` (ou le script `scripts/run_ingestion.py`) pour peupler MongoDB. La collection de chunks (par exemple, `arxiv_chunks_notebook_test` ou celle par défaut utilisée lors de l'ingestion) doit contenir des embeddings générés par le fournisseur configuré via `DEFAULT_EMBEDDING_PROVIDER` dans vos paramètres (par exemple, Hugging Face, Ollama, ou OpenAI).
* **Configuration pour l'Embedding des Requêtes :** Ce notebook utilise `RetrievalEngine`, qui vectorise les requêtes de recherche en utilisant le fournisseur d'embedding défini par la variable `DEFAULT_EMBEDDING_PROVIDER` dans vos configurations (`.env` ou `config/settings.py`).
    * Si `DEFAULT_EMBEDDING_PROVIDER` est `"openai"` : assurez-vous que `OPENAI_API_KEY` est configurée dans votre fichier `.env`.
    * Si `DEFAULT_EMBEDDING_PROVIDER` est `"huggingface"` (utilisant des modèles Sentence Transformers locaux) : aucune clé API spécifique n'est généralement requise pour l'embedding des requêtes.
    * Si `DEFAULT_EMBEDDING_PROVIDER` est `"ollama"` : assurez-vous que `OLLAMA_BASE_URL` est correctement configuré dans `.env` et que le modèle d'embedding spécifié (par exemple, `OLLAMA_EMBEDDING_MODEL_NAME`) est disponible et servi par votre instance Ollama.
* Votre instance MongoDB doit être accessible via le `MONGODB_URI` configuré dans votre fichier `.env`.

In [ ]:
import logging
import sys
from pathlib import Path
import os
import json # Pour afficher les métadonnées de manière lisible
from typing import Optional, List # 'List' était déjà là, 'Optional' est bon à avoir.

project_root = Path()

from dotenv import load_dotenv
# De même, si CWD est la racine du projet, dotenv_path serait `Path().resolve() / ".env"`
dotenv_path = project_root / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION: Fichier .env non trouvé à {dotenv_path}. Assurez-vous qu'il est à la racine du projet.")

from config.settings import settings
from config.logging_config import setup_logging
from src.rag.retrieval_engine import RetrievalEngine, RetrievedNode
from src.vector_store.mongodb_manager import MongoDBManager 

setup_logging(level="INFO") 
logger = logging.getLogger("nb_02_rag_exploration")

# --- Vérification des prérequis pour l'embedding (logique existante conservée) ---
active_embedding_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
logger.info(f"Ce notebook utilisera le fournisseur d'embedding configuré : '{active_embedding_provider}' pour RetrievalEngine.")

if active_embedding_provider == "openai":
    if not settings.OPENAI_API_KEY:
        logger.error("ERREUR : Le fournisseur d'embedding est 'openai', mais OPENAI_API_KEY n'est pas configurée. RetrievalEngine (pour l'embedding des requêtes) échouera.")
elif active_embedding_provider == "ollama":
    if not settings.OLLAMA_BASE_URL:
        logger.error("ERREUR : Le fournisseur d'embedding est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée. RetrievalEngine échouera.")
    if not settings.OLLAMA_EMBEDDING_MODEL_NAME: # Vérification déjà présente
         logger.error("ERREUR : Le fournisseur d'embedding est 'ollama', mais OLLAMA_EMBEDDING_MODEL_NAME n'est pas configuré.")
# Pour "huggingface", aucune clé API n'est généralement requise pour les modèles SentenceTransformers locaux.

if not settings.MONGODB_URI:
    logger.error("ERREUR : MONGODB_URI non trouvé. Le RetrievalEngine ne pourra pas se connecter à la base de données.")

# --- Configuration des noms de Collection et d'Index MongoDB ---
# MODIFIÉ : Pour utiliser explicitement les noms de la collection de test du notebook 01
# si l'objectif est de tester sur ces données spécifiques.
# Sinon, pour tester sur la collection principale, utilisez MongoDBManager.DEFAULT_CHUNK_COLLECTION_NAME, etc.
# Le notebook 01 définit localement :
# COLLECTION_NAME_NOTEBOOK = "arxiv_chunks_notebook_test"
# VECTOR_INDEX_NAME_NOTEBOOK = "vector_index_notebook_test"
# TEXT_INDEX_NAME_NOTEBOOK = "text_index_notebook_test"

COLLECTION_NAME_FOR_RAG_TEST = "arxiv_chunks_notebook_test"
VECTOR_INDEX_NAME_FOR_RAG_TEST = "vector_index_notebook_test"
TEXT_INDEX_NAME_FOR_RAG_TEST = "text_index_notebook_test" 

# Alternative, si vous voulez toujours un fallback vers les valeurs par défaut principales
# et que vous définissiez `settings.COLLECTION_NAME_NOTEBOOK_TEST` dans `config/settings.py`:
# COLLECTION_NAME_FOR_RAG_TEST = getattr(settings, 'COLLECTION_NAME_NOTEBOOK_TEST', MongoDBManager.DEFAULT_CHUNK_COLLECTION_NAME)
# VECTOR_INDEX_NAME_FOR_RAG_TEST = getattr(settings, 'VECTOR_INDEX_NAME_NOTEBOOK_TEST', MongoDBManager.DEFAULT_VECTOR_INDEX_NAME)
# TEXT_INDEX_NAME_FOR_RAG_TEST = getattr(settings, 'TEXT_INDEX_NAME_NOTEBOOK_TEST', MongoDBManager.DEFAULT_TEXT_INDEX_NAME)


logger.info(f"Ce notebook utilisera la collection MongoDB: '{COLLECTION_NAME_FOR_RAG_TEST}'")
logger.info(f"Index vectoriel supposé: '{VECTOR_INDEX_NAME_FOR_RAG_TEST}'")
logger.info(f"Index textuel supposé (pour hybride): '{TEXT_INDEX_NAME_FOR_RAG_TEST}'")

Variables d'environnement chargées depuis : .env
2025-06-03 10:03:15 - nb_02_rag_exploration - INFO - Ce notebook utilisera le fournisseur d'embedding configuré : 'ollama' pour RetrievalEngine.
2025-06-03 10:03:15 - nb_02_rag_exploration - INFO - Ce notebook utilisera la collection MongoDB: 'arxiv_chunks_notebook_test'
2025-06-03 10:03:15 - nb_02_rag_exploration - INFO - Index vectoriel supposé: 'vector_index_notebook_test'
2025-06-03 10:03:15 - nb_02_rag_exploration - INFO - Index textuel supposé (pour hybride): 'text_index_notebook_test'


### Initialisation du `RetrievalEngine`

Nous allons d'abord créer une instance de notre `RetrievalEngine`. Il se connectera à MongoDB et chargera l'index vectoriel existant. Assurez-vous que la collection et l'index spécifiés existent et contiennent des données.

In [2]:
retrieval_engine_instance: Optional[RetrievalEngine] = None
try:
    retrieval_engine_instance = RetrievalEngine(
        collection_name=COLLECTION_NAME_FOR_RAG_TEST,
        vector_index_name=VECTOR_INDEX_NAME_FOR_RAG_TEST
        # text_key et embedding_field sont pris par défaut par RetrievalEngine ou settings
    )
    logger.info("RetrievalEngine initialisé avec succès.")
except Exception as e:
    logger.error(f"Erreur lors de l'initialisation du RetrievalEngine: {e}", exc_info=True)
    print(f"ERREUR: Impossible d'initialiser RetrievalEngine. Vérifiez les logs et la configuration (API Keys, MongoDB URI, nom de la collection/index).")

# Petite fonction pour afficher les résultats de manière lisible
def print_retrieved_nodes(nodes: List[RetrievedNode], query: str):
    print(f"\n--- Résultats de la recherche pour la requête : '{query}' ---")
    if not nodes:
        print("Aucun document pertinent trouvé.")
        return
    for i, node in enumerate(nodes):
        print(f"\nRésultat # {i+1}:")
        print(f"  Score        : {node.score:.4f}" if node.score is not None else "  Score        : N/A")
        print(f"  Chunk ID     : {node.metadata.get('chunk_id', 'N/A')}")
        print(f"  ArXiv ID     : {node.metadata.get('arxiv_id', 'N/A')}")
        print(f"  Titre Doc    : {node.metadata.get('original_document_title', 'N/A')}")
        # Afficher d'autres métadonnées si elles sont intéressantes
        # print(f"  Autres Meta  : {json.dumps({k: v for k, v in node.metadata.items() if k not in ['chunk_id', 'arxiv_id', 'original_document_title']}, indent=2)}")
        print(f"  Texte        : {node.text[:300].replace(chr(10), ' ')}...") # Extrait du texte
    print("--------------------------------------------------")

2025-06-03 10:03:15 - src.rag.retrieval_engine - INFO - Configuring LlamaIndex global embed_model for provider: ollama
2025-06-03 10:03:15 - src.rag.retrieval_engine - INFO - LlamaIndex embed_model configured for Ollama: model='nomic-embed-text', base_url='http://localhost:11434'
2025-06-03 10:03:16 - src.rag.retrieval_engine - INFO - MongoDBAtlasVectorSearch store configured for collection 'arxiv_chunks_notebook_test'.
2025-06-03 10:03:17 - src.rag.retrieval_engine - INFO - VectorStoreIndex loaded from MongoDB store.
2025-06-03 10:03:17 - src.rag.retrieval_engine - INFO - Default retriever configured.
2025-06-03 10:03:17 - src.rag.retrieval_engine - INFO - RetrievalEngine initialized with LlamaIndex components.
2025-06-03 10:03:17 - nb_02_rag_exploration - INFO - RetrievalEngine initialisé avec succès.


### Définition des Requêtes d'Exemple

Définissons quelques requêtes pertinentes pour notre corpus (supposé être sur "Reinforcement Learning for Robotics" ou "Explainable AI for Robotics" si vous avez utilisé la query du notebook 01). Adaptez ces requêtes si votre corpus est différent.

In [3]:
sample_queries = [
    "What are the main challenges in applying reinforcement learning to robotic manipulation?",
    "Explainable AI techniques for robot decision making.",
    "How can sim-to-real transfer be improved for RL agents in robotics?",
    "Common algorithms for path planning in multi-robot systems using RL."
]
# Choisir une requête pour les tests
test_query = sample_queries[0] 
logger.info(f"Requête de test sélectionnée : '{test_query}'")

2025-06-03 10:03:17 - nb_02_rag_exploration - INFO - Requête de test sélectionnée : 'What are the main challenges in applying reinforcement learning to robotic manipulation?'


### Stratégie 1: Recherche Vectorielle Simple

Nous utilisons la méthode `retrieve_simple_vector_search` de notre `RetrievalEngine` pour effectuer une recherche basée sur la similarité sémantique.

In [4]:
if retrieval_engine_instance:
    logger.info(f"\nExécution de la recherche vectorielle simple pour : '{test_query}'")
    simple_search_results = retrieval_engine_instance.retrieve_simple_vector_search(
        query_text=test_query,
        top_k=3 
    )
    print_retrieved_nodes(simple_search_results, test_query)
else:
    logger.warning("RetrievalEngine non initialisé. Test de recherche vectorielle simple sauté.")

2025-06-03 10:03:17 - nb_02_rag_exploration - INFO - 
Exécution de la recherche vectorielle simple pour : 'What are the main challenges in applying reinforcement learning to robotic manipulation?'
2025-06-03 10:03:17 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
2025-06-03 10:03:17 - src.rag.retrieval_engine - INFO - Retrieved 3 nodes for query: 'What are the main challenges in applying reinforce...' with top_k=3 and filters=None

--- Résultats de la recherche pour la requête : 'What are the main challenges in applying reinforcement learning to robotic manipulation?' ---

Résultat # 1:
  Score        : 0.7929
  Chunk ID     : 2505.24878_chunk_015
  ArXiv ID     : 2505.24878
  Titre Doc    : Open CaptchaWorld: A Comprehensive Web-based Platform for Testing and Benchmarking Multimodal LLM Agents
  Texte        : , Yi Dong, Jieyu Zhang, Jan Kautz, Bryan Catanzaro, Andrew Tao, Qingyun Wu, Zhiding Yu, and Guilin Liu. Nemotron-research-tool-n1: T

### Stratégie 2: Recherche Vectorielle avec Filtres de Métadonnées

LlamaIndex et MongoDB Atlas Vector Search permettent de filtrer les résultats en fonction des métadonnées associées aux chunks. Notre `RetrievalEngine` expose cette fonctionnalité.

Supposons que nous voulons rechercher des informations sur notre `test_query`, mais uniquement dans des documents avec un `arxiv_id` spécifique ou une catégorie principale particulière.

**Note:** Pour que ce test soit significatif, vous devez connaître des `arxiv_id` ou des catégories présentes dans votre base de données (celles de la collection `{COLLECTION_NAME_FOR_RAG_TEST}`). Vous pouvez les trouver en explorant les résultats de la recherche simple ci-dessus ou directement dans MongoDB.

In [5]:
if retrieval_engine_instance:
    # Exemple 1: Filtrer par un arxiv_id spécifique
    # Remplacez 'example_arxiv_id_123' par un ID ArXiv réel de votre base de données
    # Vous pouvez obtenir un ID à partir des résultats de la recherche simple précédente.
    target_arxiv_id = None
    if simple_search_results and simple_search_results[0].metadata.get("arxiv_id"):
        target_arxiv_id = simple_search_results[0].metadata.get("arxiv_id")
        logger.info(f"Utilisation de l'arxiv_id '{target_arxiv_id}' pour le test de filtre (provenant du premier résultat de la recherche précédente).")
    else:
        # Mettez un ID connu de votre base de données si la recherche précédente n'a rien donné
        logger.warning("Aucun arxiv_id récupéré de la recherche précédente. Le filtre par ID pourrait ne pas fonctionner sans un ID valide.")
        # target_arxiv_id = "your_known_arxiv_id_here" # Décommentez et remplacez

    if target_arxiv_id:
        logger.info(f"\nRecherche vectorielle pour '{test_query}' AVEC filtre sur arxiv_id='{target_arxiv_id}'")
        filtered_results_by_id = retrieval_engine_instance.retrieve_simple_vector_search(
            query_text=test_query,
            top_k=2,
            metadata_filters=[{"key": "arxiv_id", "value": target_arxiv_id}] # Clé telle que stockée dans MongoDB
        )
        print_retrieved_nodes(filtered_results_by_id, f"{test_query} (filtré par arxiv_id: {target_arxiv_id})")
    else:
        logger.info("Saut du test de filtre par arxiv_id car target_arxiv_id n'est pas défini.")

    # Exemple 2: Filtrer par catégorie principale (si stockée et indexée)
    # Supposons que votre index vectoriel a été créé avec "metadata.primary_category" comme champ de filtre.
    # Et que vos documents MongoDB ont un champ `metadata: { "primary_category": "cs.RO", ... }`
    # Notre `MongoDBManager` a été configuré pour créer un filtre sur "metadata.primary_category".
    # Et `RetrievalEngine` doit passer la clé de filtre correctement (LlamaIndex gère les chemins de points).
    target_category = "cs.RO" # Exemple: Robotics
    logger.info(f"\nRecherche vectorielle pour '{test_query}' AVEC filtre sur primary_category='{target_category}'")
    # La clé de filtre pour LlamaIndex doit correspondre au chemin exact dans le document MongoDB
    # Si dans MongoDB c'est {"metadata": {"primary_category": "cs.RO"}}, la clé est "metadata.primary_category"
    # Si c'est {"primary_category": "cs.RO"} au premier niveau, la clé est "primary_category"
    # Nos chunks stockent `metadata` comme un dict imbriqué, donc "metadata.primary_category" est correct.
    filtered_results_by_category = retrieval_engine_instance.retrieve_simple_vector_search(
        query_text=test_query,
        top_k=2,
        metadata_filters=[{"key": "metadata.primary_category", "value": target_category}]
    )
    print_retrieved_nodes(filtered_results_by_category, f"{test_query} (filtré par catégorie: {target_category})")

else:
    logger.warning("RetrievalEngine non initialisé. Tests de recherche avec filtres sautés.")

2025-06-03 10:03:17 - nb_02_rag_exploration - INFO - Utilisation de l'arxiv_id '2505.24878' pour le test de filtre (provenant du premier résultat de la recherche précédente).
2025-06-03 10:03:17 - nb_02_rag_exploration - INFO - 
Recherche vectorielle pour 'What are the main challenges in applying reinforcement learning to robotic manipulation?' AVEC filtre sur arxiv_id='2505.24878'
2025-06-03 10:03:17 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
2025-06-03 10:03:17 - src.rag.retrieval_engine - INFO - Retrieved 2 nodes for query: 'What are the main challenges in applying reinforce...' with top_k=2 and filters=[{'key': 'arxiv_id', 'value': '2505.24878'}]

--- Résultats de la recherche pour la requête : 'What are the main challenges in applying reinforcement learning to robotic manipulation? (filtré par arxiv_id: 2505.24878)' ---

Résultat # 1:
  Score        : 0.7929
  Chunk ID     : 2505.24878_chunk_015
  ArXiv ID     : 2505.24878
  Titre D

### Stratégie 3: Recherche Hybride (Vectorielle + Textuelle) - Discussion

Notre `MongoDBManager` a la capacité de créer à la fois des index vectoriels et des index de recherche textuelle (Atlas Search). LlamaIndex, via `MongoDBAtlasVectorSearch`, peut également être configuré pour utiliser ces deux types d'index pour effectuer une recherche hybride.

**Configuration pour la Recherche Hybride avec LlamaIndex:**
1.  **Index MongoDB :** Assurez-vous que vous avez :
    * Un index de recherche vectorielle (par exemple, `default_vector_index`) sur le champ d'embedding.
    * Un index de recherche textuelle Atlas Search (par exemple, `default_text_index`) sur les champs textuels pertinents (comme `text_chunk`, `original_document_title`).
    Ces index sont créés par notre `MongoDBManager` si vous exécutez `run_ingestion.py`.

2.  **`MongoDBAtlasVectorSearch` de LlamaIndex :** Lors de son initialisation, vous pouvez spécifier le `fulltext_index_name`.
    ```python
    # Exemple d'initialisation (pas exécuté ici, juste pour illustration)
    # vector_store_for_hybrid = MongoDBAtlasVectorSearch(
    #     uri=settings.MONGODB_URI,
    #     db_name=settings.MONGO_DATABASE_NAME,
    #     collection_name=COLLECTION_NAME_FOR_RAG_TEST,
    #     index_name=VECTOR_INDEX_NAME_FOR_RAG_TEST, # Index vectoriel
    #     fulltext_index_name=TEXT_INDEX_NAME_FOR_RAG_TEST, # Index textuel !
    #     embedding_key="embedding",
    #     text_key="text_chunk"
    # )
    # index_for_hybrid = VectorStoreIndex.from_vector_store(vector_store_for_hybrid)
    ```

3.  **Récupérateur LlamaIndex :** Vous pouvez ensuite obtenir un récupérateur ou un moteur de requête en mode hybride.
    ```python
    # Exemple de récupérateur hybride (pas exécuté ici)
    # hybrid_retriever = index_for_hybrid.as_retriever(
    #     vector_store_query_mode="hybrid",
    #     similarity_top_k=3, # Nombre de résultats de la recherche vectorielle
    #     fulltext_top_n=3,   # Nombre de résultats de la recherche textuelle
    #     alpha=0.5           # Pondération (0.0 = textuel seul, 1.0 = vectoriel seul)
    # )
    # results = hybrid_retriever.retrieve("ma requête")
    
    # Ou avec un QueryEngine :
    # query_engine_hybrid = index_for_hybrid.as_query_engine(
    #     vector_store_query_mode="hybrid", 
    #     similarity_top_k=3, 
    #     alpha=0.5
    # )
    # response = query_engine_hybrid.query("ma requête")
    ```

Notre classe `RetrievalEngine` actuelle ne possède pas de méthode dédiée `retrieve_hybrid_search`, mais elle pourrait être étendue pour en inclure une en utilisant la configuration ci-dessus. Pour l'instant, si les deux index existent dans MongoDB, la base est là pour l'implémenter.

Le notebook `GenAI-Showcase/notebooks/rag/retrieval_strategies_mongodb_llamaindex.ipynb` fourni en référence montre en détail comment utiliser ces modes avec LlamaIndex.

### Futures Explorations : Stratégies RAG Avancées

LlamaIndex offre de nombreuses autres stratégies RAG avancées qui pourraient être explorées dans des notebooks ultérieurs ou intégrées dans notre `RetrievalEngine` :
* **Parent Document Retriever** : Récupère des chunks plus petits mais retourne les documents parents plus larges pour un meilleur contexte.
* **HyDE (Hypothetical Document Embeddings)** : Génère un document hypothétique en réponse à la requête, embedde ce document, puis utilise cet embedding pour la recherche.
* **Self-Querying Retriever** : Utilise un LLM pour convertir une requête en langage naturel en une requête structurée qui inclut des filtres de métadonnées.
* **Et bien d'autres...**

Ces techniques peuvent améliorer significativement la pertinence et la qualité des informations fournies aux agents LLM.

## Conclusion de l'Exploration RAG

Ce notebook a démontré comment utiliser notre `RetrievalEngine` pour effectuer des recherches vectorielles simples et des recherches avec filtres de métadonnées. Nous avons également discuté de la manière dont la recherche hybride et d'autres stratégies RAG avancées pourraient être mises en œuvre avec LlamaIndex et notre backend MongoDB.

Les prochaines étapes pourraient impliquer l'enrichissement du `RetrievalEngine` avec ces stratégies plus avancées ou l'utilisation de ce moteur de récupération par nos agents LangGraph.